In [ ]:
import sys
import os
import time

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)
time.sleep(2)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import argparse
import json

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import set_random_seed

import keras.backend as K
from keras.backend.tensorflow_backend import set_session

from core.data.data_loader import *
from core.models.lstm import ModelLSTM, ModelLSTMParalel, ModelLSTMCuDnnParalel
from core.models.cnn import ModelCNN
from core.models.mlp import ModelMLP

from core.utils.metrics import *
from core.utils.utils import *


## loading Configurartion ##


In [ ]:
# load some configurations configurations of model and others
configs = json.load(open('config_lstm.json', 'r'))
output_path = configs['paths']['save_dir']
output_logs = configs['paths']['log_dir']

if os.path.isdir(output_path) == False:
    os.mkdir(output_path)

if os.path.isdir(output_logs) == False:
    os.mkdir(output_logs)        

time_steps =  configs['model']['layers'][0]['input_timesteps']  # the number of points or hits
num_features = configs['model']['layers'][0]['input_features']  # the number of features of each hits

split = configs['data']['train_split']  # the number of features of each hits
cylindrical = configs['data']['cylindrical']  # set to polar or cartesian coordenates
normalise = configs['data']['normalise'] 
num_hits = configs['data']['num_hits']
print('OK reading of json file')

## Setup Data ##

In [ ]:
# reading path of data
path = configs['data']['filename']
path

In [ ]:
data = Dataset(path, split, cylindrical, num_hits, KindNormalization.Zscore)


In [ ]:
X_train, y_train = data.get_training_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=True)
#dataset = dataset.iloc[0:2640,0:]/   b
#dataset = dataset.iloc[0:31600,0:]
print(X_train.shape)
print(y_train.shape)
X_train.head(6)

In [ ]:
# reshape data     
time_steps =  configs['model']['layers'][0]['input_timesteps']  # the number of points or hits
num_features = configs['model']['layers'][0]['input_features']  # the number of features of each hits
split = configs['data']['train_split']  # the number of features of each hits
cylindrical = configs['data']['cylindrical']  # set to polar or cartesian coordenates
normalise = configs['data']['normalise'] 

In [ ]:
# there is empty data?
print(X_train.isnull().values.any())
#print(X_test.isnull().values.any())



In [ ]:
X_train = data.reshape3d(X_train, time_steps, num_features)

print('[Data] shape X_train ', X_train.shape)
print('[Data] shape y_train ', y_train.shape)


In [ ]:
def manage_models(config):
    
    type_model = config['model']['name']
    model = None

    if type_model == 'lstm': #simple LSTM
        model = ModelLSTM(config)
    elif type_model == 'lstm-paralel':
        model = ModelLSTMParalel(config)
    elif type_model == 'cnn':
        model = ModelCNN(config)
    elif type_model == 'mlp':
        model = ModelMLP(config)        

    return model


## Create Model ##

In [ ]:
# geramos o modelo
#configs = json.load(open('config_test.json', 'r'))
model = manage_models(configs)

if model is None:
    print('Please instance model')

loadModel = configs['training']['load_model']
#loadModel = False
#loadModel = True 

## Create Model ##

In [ ]:
from core.utils.metrics import *

# in-memory training
if loadModel == False:
    model.build_model()
    # in-memory training
    history = model.train(
        x=X_train,
        y=y_train,
        epochs = configs['training']['epochs'],
        batch_size = configs['training']['batch_size']
    )

    evaluate_training(history, output_path)

elif loadModel == True:       
    if not model.load_model():
        print ('[Error] please change the config file : load_model')

## Evaluate The model (just training)##

In [ ]:
#evaluate the model
from core.utils.metrics import *
#evaluate_training(history,output_path)


## Normal Prediction and Nearest ##
We concatenate features to predict one hit.

In [ ]:
# Prediction
import numpy as np
from core.utils.metrics import *
from core.utils.utils import *

In [ ]:
# divimos 
X_test, y_test = data.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=True)
print(X_test.shape)
print(y_test.shape)
X_test.head(6)

### Normal Prediction ###

In [ ]:
X_test_ = data.reshape3d(X_test, time_steps, num_features)
y_tes_ = data.reshape3d(y_test, 6, num_features)

pred_full = model.predict_full_sequences(X_test_, y_tes_, 6)
print('finished prediction ...')

predicted = convert_vector_to_matrix(pred_full, 3, 6)
predicted = to_frame(predicted)
predicted.head()

### Adding Nearest ###

In [ ]:
# convertimos a matriz do test em um vetor 
#y_test_ = convert_matrix_to_vec(y_test, 3)
#y_test_ = np.array(y_test_)

pred_full_res, correct = model.predict_full_sequences_nearest(X_test_, y_test, 6)
print('finished prediction nearest...')

predicted_nearest = convert_vector_to_matrix(pred_full_res, 3, 6)
predicted_nearest = to_frame(predicted_nearest)
predicted_nearest.head()


In [ ]:
# vetor de contagens
total = X_test.shape[0]

print(correct)

#taxa_acertos = [(correct*100)/total for x in range(0, len(correct))]
print((correct*100)/total)

### Metrics for Normal Prediction ###

In [ ]:
# metricas sem nearest
result = calc_score(data.reshape2d(y_test, 1),
                    data.reshape2d(predicted, 1), report=False)
print(result)
r2, rmse, rmses = evaluate_forecast_seq(y_test, predicted)
summarize_scores(r2, rmse, rmses)


### Metrics for  Nearest Prediction ###

In [ ]:
# metricas para nearest
result = calc_score(data.reshape2d(y_test, 1),
                    data.reshape2d(predicted_nearest, 1), report=False)
print(result)
r2, rmse, rmses = evaluate_forecast_seq(y_test, predicted_nearest)
summarize_scores(r2, rmse, rmses)

### Convert to Millimeters ###

In [ ]:
if normalise:
    print('[Data] Inverse Transforming ...')
    y_test_orig = data.inverse_transform_y(y_test)
    y_predicted_orig = data.inverse_transform_y(predicted)
    y_predicted_orig_nearest = data.inverse_transform_y(predicted_nearest)
    
else:
    y_test_orig = y_test
    y_predicted_orig = predicted
    y_predicted_orig_nearest = predicted_nearest


y_test_orig = pd.DataFrame(y_test_orig)
y_predicted_orig = pd.DataFrame(y_predicted_orig)
y_predicted_orig_nearest = pd.DataFrame(y_predicted_orig_nearest)

print('[Data] shape y_predicted_orig ', y_predicted_orig.shape)    
print('[Data] shape y_test_orig ', y_test_orig.shape)

result = calc_score(data.reshape2d(y_test_orig, 1),
                    data.reshape2d(y_predicted_orig_nearest, 1), report=False)
print(result)
r2, rmse, rmses = evaluate_forecast_seq(y_test_orig, y_predicted_orig_nearest)
summarize_scores(r2, rmse, rmses)


## Prepare data for Visualization ##

In [ ]:
x_true, y_true = data.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=False)
#x_true = X_test
#y_true = y_test
print(x_true.shape)
print(y_true.shape)

In [ ]:
# join both dataset must be equal to X_test
data_test = np.concatenate([x_true , y_true], axis = 1)
data_test = pd.DataFrame(data_test)
print('Shape ', data_test.shape)

In [ ]:
# must be equal
#data_test.values == X_test_new.values


In [ ]:
# join both dataset should be equal to X_test but not equal
data_pred = np.concatenate([x_true , y_predicted_orig], axis = 1 )
data_pred = pd.DataFrame(data_pred)
print('Shape ', data_pred.shape)

In [ ]:
# join both dataset should be equal to X_test but not equal
data_pred_nearest = np.concatenate([x_true , y_predicted_orig_nearest], axis = 1 )
data_pred_nearest = pd.DataFrame(data_pred_nearest)
print('Shape ', data_pred_nearest.shape)

In [ ]:
# should not be equal
#data_pred.values == X_test_new.values

In [ ]:
if cylindrical:

    y_test_orig.to_csv(os.path.join(output_path, 'y_true_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    y_predicted_orig.to_csv(os.path.join(output_path, 'y_pred_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    x_true.to_csv(os.path.join(output_path, 'x_true_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)
else:
    
    y_test_orig.to_csv(os.path.join(output_path, 'y_true_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)
    y_predicted_orig.to_csv(os.path.join(output_path, 'y_pred_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    x_true.to_csv(os.path.join(output_path, 'x_true_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)

print('[Output] Results saved at %', output_path)

## Visualization ##

In [ ]:
# Dados originais
N_tracks = 50
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-original-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-original-%s_xyz_nearest.html' % name)
    
fig = track_plot_xyz([data_test], n_hits = 10, cylindrical = cylindrical, n_tracks = 50, 
               path=path_html, title='Track Original #10 Hit - Model %s' % name.upper())

fig.show()

In [ ]:
# predicted data
N_tracks = 30
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-predicted-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-predicted-%s_xyz_nearest.html' % name)
    
fig = track_plot_xyz([data_test, data_pred], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Prediction #10 Hit - Model %s (Nearest hits)' % name.upper())

fig.show()

In [ ]:
# predicted data
N_tracks = 3
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-predicted-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-predicted-%s_xyz_nearest.html' % name)

data_test.name = 'real' 
data_pred.name = 'predito'
data_pred_nearest.name = 'nearest'
fig = track_plot_xyz([data_test, data_pred, data_pred_nearest], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Prediction #10 Hit - Model %s (Nearest hits)' % name.upper(), auto_open=True)

fig.show()